In [ ]:
%matplotlib inline  
# To make data visualisations display in Jupyter Notebooks
import numpy as np   # linear algebra
import pandas as pd  # Data processing, Input & Output load
import matplotlib.pyplot as plt # Visuvalization & plotting
import seaborn as sns # Also for Data visuvalization

from sklearn.metrics.pairwise import cosine_similarity  # Compute cosine similarity between samples in X and Y.
from scipy import sparse  #  sparse matrix package for numeric data.
from scipy.sparse.linalg import svds # svd algorithm

import warnings   # To avoid warning messages in the code run
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [ ]:
Rating = pd.read_csv('Ratings.csv')
Movie_D = pd.read_csv('Movie details.csv',encoding='latin-1') ##Movie details

In [ ]:
Rating.shape

In [ ]:
Rating.head()

In [ ]:
Rating.columns = ['user_id', 'movie_id', 'rating', 'timestamp']

In [ ]:
Movie_D.shape

In [ ]:
Movie_D.head()

In [ ]:
Movie_D.columns = ['movie_id', 'movie_title', 'release_date', 'video_release_date ',
       'IMDb_URL', 'unknown', 'Action ', 'Adventure', 'Animation',
       'Childrens', 'Comedy ', 'Crime ', ' Documentary ', 'Drama',
       ' Fantasy', 'Film-Noir ', 'Horror ', 'Musical', 'Mystery',
       ' Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

In [ ]:
Movie_Rating = pd.merge(Rating, Movie_D, on='movie_id')
Movie_Rating.describe()

In [ ]:
n_users = Movie_Rating.user_id.unique().shape[0]
n_items = Movie_Rating.movie_id.unique().shape[0]
print(n_items, n_users)

In [ ]:
# Calculate mean rating of all movies 
Movie_Stats = pd.DataFrame(Movie_Rating.groupby('movie_title')['rating'].mean())
Movie_Stats.sort_values(by = ['rating'],ascending=False).head()

In [ ]:
# Calculate count rating of all movies
Movie_Stats['Count_of_ratings'] = pd.DataFrame(Movie_Rating.groupby('movie_title')['rating'].count())
Movie_Stats.sort_values(by =['Count_of_ratings'], ascending=False).head()

In [ ]:
Movie_Stats['Count_of_ratings'].hist(bins=50)

In [ ]:
sns.jointplot(x='rating', y='Count_of_ratings', data=Movie_Stats)

In [ ]:
User_movie_Rating = Movie_Rating.pivot_table(index='user_id', columns='movie_title', values='rating')
User_movie_Rating.head()

In [ ]:
User_movie_Rating.shape

In [ ]:
User_movie_Rating['Air Force One (1997)']

In [ ]:
Similarity = User_movie_Rating.corrwith(User_movie_Rating['Air Force One (1997)'])
Similarity.head()

In [ ]:
corr_similar = pd.DataFrame(Similarity, columns=['Correlation'])
corr_similar.sort_values(['Correlation'], ascending= False).head(10)

In [ ]:
corr_similar_num_of_rating = corr_similar.join(Movie_Stats['Count_of_ratings'])
corr_similar_num_of_rating.sort_values(['Correlation'], ascending= False).head(10)

In [ ]:
corr_similar_num_of_rating[corr_similar_num_of_rating ['Count_of_ratings']>50].sort_values('Correlation', ascending=False).head()

In [ ]:
Movie_cosine = Movie_Rating[['user_id','movie_id','rating']]
Movie_cosine.head()

In [ ]:
data = Movie_cosine.rating
col = Movie_cosine.movie_id
row = Movie_cosine.user_id

R = sparse.coo_matrix((data, (col, row))).tocsr()
print('{0}x{1} user by movie matrix'.format(*R.shape))

In [ ]:
find_similarities = cosine_similarity(R.T) # We are transposing the matrix 
print (find_similarities.shape)

In [ ]:
find_similarities[4].argsort()

In [ ]:
find_similarities[4].argsort()[-5:]

In [ ]:
Movie_D.iloc[4]

In [ ]:
Movie_D.iloc[find_similarities[4].argsort()[-5:]]

In [ ]:
df_matrix = User_movie_Rating.fillna(0)

In [ ]:
df_matrix.shape

In [ ]:
df_matrix.iloc[2]

In [ ]:
U, SIGMA, VT = svds(df_matrix) #k=20
U.shape, SIGMA.shape, VT.shape

In [ ]:
SIGMA = np.diag(SIGMA)
U.shape, SIGMA.shape, VT.shape

In [ ]:
reconstructed_df = np.dot(np.dot(U, SIGMA), VT)
predictions_df = pd.DataFrame(reconstructed_df, columns = df_matrix.columns)
predictions_df.iloc[2]

In [ ]:
predictions_df.iloc[2][df_matrix.iloc[2]==0.0].sort_values(ascending=False).head()